In [2]:
import json

class CharacterTurtleGenerator:
    """
    基于 MakeMeAHanzi 项目的 graphics.txt 数据，生成汉字笔画的 Turtle 绘图代码或可视化。

    --- graphics.txt 数据说明 ---
    数据来源: MakeMeAHanzi 项目 (https://github.com/skishore/makemeahanzi)
    文件格式: 每行一个 JSON 对象，描述一个汉字。

    每个 JSON 对象包含以下关键字段:

    1. strokes (轮廓数据)
       标准 SVG Path 指令，定义了每个笔画的外边界（空心字轮廓）。
       常见指令:
         - M (MoveTo):   移动到坐标点（不画线）
         - L (LineTo):   画直线到目标点
         - Q (Quadratic Bézier Curve): 二次贝塞尔曲线
         - C (Cubic Bézier Curve):     三次贝塞尔曲线
         - Z (ClosePath): 闭合路径

    2. medians (骨架/中轴线数据) ★ 核心字段
       嵌套列表，每个子列表代表一个笔画，由一系列 [x, y] 坐标点组成。
       顺次连接这些点即为该笔画的中心线（骨架）。
       坐标范围约 0~1024, 点的顺序符合汉字笔顺。

       示例: [[140,741],[175,713], ... ,[51,73]] 表示一个笔画的中轴线。

    本类的核心思路:
      忽略 strokes(轮廓), 直接解析 medians(骨架)。
      因为 medians 已将曲线离散化为点集，只需让 Turtle 在点集之间 goto 即可绘制。
    """

    def __init__(self, file_path):
        """加载 graphics.txt 数据（每行一个 JSON,解析 medians 骨架坐标）"""
        self.data_map = {}
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                item = json.loads(line)
                self.data_map[item['character']] = item

    def _get_coordinates(self, char, scale=0.4, offset=(250, 250)):
        """
        获取并转换坐标。
        注意:ColabTurtle 原点在左上角 (0,0)，画布通常是 500x500 或 800x600。
        """
        if char not in self.data_map:
            return None
        
        medians = self.data_map[char]['medians']
        processed_strokes = []
        
        for stroke in medians:
            new_stroke = []
            for x, y in stroke:
                # 原始数据范围约 0-1024
                # 我们将其缩放并平移到画布可视区域
                tx = x * scale + offset[0] - (512 * scale)
                ty = y * scale + offset[1] - (512 * scale)
                new_stroke.append((round(tx, 1), round(ty, 1)))
            processed_strokes.append(new_stroke)
        return processed_strokes

    def to_python_code(self, char, function_name=None):
        """输出纯文本代码，用于喂给 LLM 学习"""
        strokes = self._get_coordinates(char, scale=1.0, offset=(0,0)) # 原始比例输出
        if not strokes: return f"# 字符 '{char}' 不在库中"
        
        fn = function_name or f"draw_{char}"
        lines = [f"def {fn}(t):"]
        
        for i, stroke in enumerate(strokes):
            lines.append(f"    # 笔划 {i+1}")
            lines.append(f"    t.penup()")
            lines.append(f"    t.goto({stroke[0][0]}, {stroke[0][1]})")
            lines.append(f"    t.pendown()")
            for x, y in stroke[1:]:
                lines.append(f"    t.goto({x}, {y})")
        return "\n".join(lines)

    def visualize_in_colab(self, char):
        """在 Colab 或 Jupyter 中可视化"""
        try:
            from ColabTurtlePlus.Turtle import Turtle, clearscreen, setup
        except ImportError:
            print("请先运行: pip install ColabTurtlePlus")
            return

        # 获取坐标并缩放到适合 200x200 画布
        strokes = self._get_coordinates(char, scale=0.2, offset=(0, 20))
        if not strokes:
            print("未找到该字")
            return
        
        # 初始化画布
        clearscreen()
        setup(200, 200) 
        
        t = Turtle()
        t.speed(8)
        t.pensize(3)
        
        for stroke in strokes:
            t.penup()
            t.goto(stroke[0][0], stroke[0][1])
            t.pendown()
            for x, y in stroke[1:]:
                t.goto(x, y)
        
        t.hideturtle()

In [3]:
gen = CharacterTurtleGenerator("graphics.txt")

In [49]:
print(gen.to_python_code("丁"))

def draw_丁(t):
    # 笔划 1
    t.penup()
    t.goto(-359.0, 133.0)
    t.pendown()
    t.goto(-335.0, 122.0)
    t.goto(-293.0, 116.0)
    t.goto(-96.0, 151.0)
    t.goto(282.0, 194.0)
    t.goto(311.0, 190.0)
    t.goto(375.0, 167.0)
    # 笔划 2
    t.penup()
    t.goto(-34.0, 132.0)
    t.pendown()
    t.goto(6.0, 98.0)
    t.goto(6.0, -411.0)
    t.goto(-17.0, -457.0)
    t.goto(-62.0, -444.0)
    t.goto(-143.0, -402.0)


In [6]:
gen.visualize_in_colab("到")